In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
import swifter
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
#pip install swifter
#nltk.download('vader_lexicon')
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [2]:
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [3]:
df=pd.read_excel('hotel_reviews (2).xlsx')

In [4]:
df.drop(['@'],inplace=True,axis=1)

In [5]:
df

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [6]:
def text_cleaning(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and extra whitespaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove punctuation
    text = ''.join([x for x in text if x not in string.punctuation])

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.update(['room', 'hotel','restaurant','pepole','day','night'])
    tokens = [x for x in tokens if x not in stop_words]
   #tokens = [x for x in tokens if x not in stopwords.words('english')]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Join the tokens back into a single string
    cleaned_text = ' '.join(tokens)

    return cleaned_text


In [7]:
df['cleaned_text']=df["Review"].swifter.apply(text_cleaning)

Pandas Apply:   0%|          | 0/20491 [00:00<?, ?it/s]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
import joblib

In [9]:
features=df['cleaned_text']
features

0        nice expens park got good deal stay anniversar...
1        ok noth special charg diamond member hilton de...
2        nice room experi monaco seattl good nt levelpo...
3        uniqu great stay wonder time monaco locat exce...
4        great stay great stay went seahawk game awesom...
                               ...                        
20486    best kept secret rd time stay charm star ca nt...
20487    great locat price view great quick place sight...
20488    ok look nice modern outsid desk staff nt parti...
20489    theft ruin vacat open sept guest week happi st...
20490    peopl talk ca nt believ excel rate nt ye patri...
Name: cleaned_text, Length: 20491, dtype: object

In [10]:
df['sentiment'] = pd.cut(df['Rating'], bins=[0, 2, 3, 5], labels=['negative', 'neutral', 'positive'])

In [11]:
target=df['sentiment']

In [12]:
pip install --upgrade scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [13]:
vectorizer=TfidfVectorizer()

In [14]:
vectorizer.fit(features)

TfidfVectorizer()

In [15]:
joblib.dump(vectorizer,'vec.joblib')

['vec.joblib']

In [16]:
new_features=vectorizer.transform(features)

In [17]:
x_train,x_test,y_train,y_test=train_test_split(new_features,target,train_size=0.7,random_state=70)

In [18]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler



# Create the RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)

# Fit and resample
x_train_resampled, y_train_resampled = oversampler.fit_resample(x_train, y_train)


In [19]:
from sklearn.naive_bayes import MultinomialNB

In [20]:
naive=MultinomialNB()

In [21]:
naive.fit(x_train_resampled, y_train_resampled)

MultinomialNB()

In [22]:
joblib.dump(naive,"naive.joblib")

['naive.joblib']

In [23]:
pred=naive.predict(x_test)

In [24]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

    negative       0.68      0.79      0.73       978
     neutral       0.27      0.55      0.36       638
    positive       0.96      0.78      0.86      4532

    accuracy                           0.76      6148
   macro avg       0.63      0.71      0.65      6148
weighted avg       0.84      0.76      0.79      6148



In [25]:
from sklearn.linear_model import LogisticRegression

In [28]:
log=LogisticRegression()

In [29]:
log.fit(x_train_resampled, y_train_resampled)

C:\Users\Hp\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
joblib.dump(log,'log.joblib')

['log.joblib']

In [32]:
sent = ["the hotel was ok"]
transformed = vectorizer.transform(sent)
prediction = log.predict(transformed)
print("Predicted Sentiment:", prediction)

Predicted Sentiment: ['neutral']
